# HR Analytics Dashboard

This dashboard is designed to analyze HR data and present key trends and metrics.
It provides interactive visualizations and functionalities for predicting critical employees using machine learning.

## Key Features:
- KPI Overviews (Salary, Satisfaction, Sick Days)
- Interactive Data Visualization (e.g., Correlations, Scatterplots)
- Dynamic File Upload and Processing
- Identification of Critical Employees based on AI Models (e.g., XGBoost)

# Configuration and Layout

## Key Components
1. **External CSS**:
   - The dashboard uses Tailwind CSS for modern styling.
   - Integration: `https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css`

2. **App Structure**:
   - Header: A prominently styled "HR Analytics Dashboard".
   - Navigation Buttons: Buttons for different pages, such as "KPIs & Trends" or "Critical Employees".
   - Footer: A fixed footer with creator details.

In [1]:
import io
import os
import joblib
import seaborn as sns
import base64
import dash
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
from io import BytesIO
from dash import dcc, html, Input, Output, State, dash_table
from src.data_loading import load_dataset
from src.data_cleaning import clean_dataset
from src.ML1_churn_prediction_best_model import preprocess_data
from src.ML1_churn_prediction_best_model import get_critical_employees
import webbrowser
from threading import Timer
from dash import Dash, html

# Google Chrome als Standardbrowser auf macOS setzen
chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"

# Chrome als neuen Browser registrieren
webbrowser.register("chrome", None, webbrowser.BackgroundBrowser(chrome_path))

# Beispielhafte Modellpath
model_paths = {
    "Random Forest": "models/random_forest_model.pkl",
    "XGBoost": "models/xgboost_model.pkl",
    "LightGBM": "models/lightgbm_model.pkl"
}

try:
    df = pd.read_csv("../data/HR_cleaned.csv", low_memory=False)
except FileNotFoundError:
    df = pd.DataFrame()

color_mapping = {
    "Active": "#1f77b4",
    "Left": "#2ca02c",
    "Retired": "#ff7f0e",
}

# App-Configuration
# Tailwind CSS
external_stylesheets = [
    "https://cdn.jsdelivr.net/npm/tailwindcss@2.2.19/dist/tailwind.min.css",
]

# App-Configuration
app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)
app.title = "HR Analytics Dashboard"


# App Layout
app.layout = html.Div(
    className="container mx-auto my-4 px-4",
    children=[
        # Header
        html.Div(
            className="text-center text-blue-600 font-bold text-4xl mb-6",
            children="HR Analytics Dashboard"
        ),

        # Buttons direkt unter dem Header
        html.Div(
            className="flex justify-center space-x-4 mb-6",
            children=[
                html.Button("KPIs & Trends", id="btn-to-page-1",
                            className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded"),
                html.Button("Analysis & Details", id="btn-to-page-2",
                            className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded"),
                html.Button("Correlation Matrix", id="btn-to-page-3",  # Neuer Button für Korrelationsmatrix
                            className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded"),
                html.Button("Critical Employees", id="btn-to-page-4",
                            className="bg-blue-500 hover:bg-blue-700 text-white font-bold py-2 px-4 rounded"),
            ],
        ),

        # Platzhalter für dynamische Inhalte (Seiteninhalt)
        html.Div(id="page-content", className="mt-4"),

        # Footer mit fixer Position
        html.Div(
            className="text-center text-gray-500 text-sm mt-6",
            children="© 2025 Patrick Witzl",
            style={
                "position": "fixed",
                "width": "100%",
                "bottom": "0",
                "left": "0",
                "backgroundColor": "white",
                "padding": "10px 0",
                "boxShadow": "0 -2px 5px rgba(0, 0, 0, 0.1)",
                "zIndex": "1000"
            }
        )
    ]
)


OSError: dlopen(/Users/Patrick/PyCharm Projects/Employee-Turnover-Prediction/.venv/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D44045CD-B874-3A27-9A61-F131D99AACE4> /Users/Patrick/PyCharm Projects/Employee-Turnover-Prediction/.venv/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file)

# Navigation Logic with Callbacks

The navigation logic is based on Dash callbacks:
- **Input**: Clicks on the navigation buttons.
- **Output**: `page-content`, which is dynamically updated based on the button clicks.

Each page is generated through functions like `render_page_1()` or `render_page_4()` to keep the content modular.

In [2]:
@app.callback(
    Output("page-content", "children"),
    [Input("btn-to-page-1", "n_clicks"),
     Input("btn-to-page-2", "n_clicks"),
     Input("btn-to-page-3", "n_clicks"),
     Input("btn-to-page-4", "n_clicks"),
    ]
)
def navigate(button_1_clicks, button_2_clicks, button_3_clicks, button_4_clicks,):
    ctx = dash.callback_context
    if not ctx.triggered:
        return render_page_1()  # Standardseite: Seite 1
    button_id = ctx.triggered[0]['prop_id'].split('.')[0]

    if button_id == "btn-to-page-1":
        return render_page_1()
    elif button_id == "btn-to-page-2":
        return render_page_2()
    elif button_id == "btn-to-page-3":
        return render_page_3_correlation_matrix()
    elif button_id == "btn-to-page-4":
        return render_page_4()

    return render_page_1()



1. **Dropdown for Year Selection**: Explain the dropdown functionality and its purpose in selecting years dynamically.
2. **Title Section**: Describe how the main dashboard title ('KPIs & Trends') is displayed.
3. **KPI Section**:
   - Specify the layout for three placeholder titles and values (Salary, Satisfaction, Sick Days).
   - Discuss how they act as placeholders for dynamic content.
4. **Totals Section**: Outline totals for 'New Hires', 'Left', and 'Retired' employees, and highlight the use of color coding for clarity.
5. **Monthly Trends Graphs**: Mention the use of graphs displaying monthly trends for 'Active' employees, and 'Left & Retired' employees.
6. **Yearly Trends Graphs**: Describe the annual trend graphs for both 'Active' and 'Retired or Left' employees.

Conclude the markdown cell with how the layout uses Dash components and Tailwind CSS for style and responsiveness.

In [3]:
def render_page_1():
    return html.Div(
        className="space-y-6",
        children=[

            # --- Dropdown for year selection ---
            html.Div(
                className="w-1/2 mx-auto",
                children=[
                    dcc.Dropdown(
                        id="year-dropdown",
                        options=[{"label": str(year), "value": year} for year in sorted(df["Year"].unique())],
                        placeholder="Select Year",
                        value=sorted(df["Year"].unique())[-1] if len(df["Year"].unique()) > 0 else None,
                        className="rounded border-gray-300"
                    )
                ],
            ),

            # --- Title section ---
            html.H2("KPIs & Trends", className="text-center text-blue-600 font-bold text-2xl"),

            # --- KPI Titles (e.g., Salary, Satisfaction, Sick Days) ---
            html.Div(
                className="grid grid-cols-3 gap-4",
                children=[
                    html.H3(id="kpi-title-salary", className="text-center text-lg text-gray-600"),
                    html.H3(id="kpi-title-satisfaction", className="text-center text-lg text-gray-600"),
                    html.H3(id="kpi-title-absence", className="text-center text-lg text-gray-600"),
                ]
            ),

            # --- KPI Values (average salary, satisfaction, sick days) ---
            html.Div(
                className="grid grid-cols-3 gap-4",
                children=[
                    html.H3(id="kpi-avg-salary", className="text-center text-lg text-gray-600"),
                    html.H3(id="kpi-avg-satisfaction", className="text-center text-lg text-gray-600"),
                    html.H3(id="kpi-avg-absence", className="text-center text-lg text-gray-600"),
                ]
            ),

            # --- Totals at the top of the page ---
            html.Div(
                className="grid grid-cols-3 gap-4",
                children=[
                    # New Hires
                    html.Div([
                        html.H3(id="sum-new-hires", style={"color": "#1f77b4"},  # Blue
                                className="text-center text-lg font-bold"),
                        html.P("New Hires", className="text-center text-gray-600")
                    ]),
                    # Left
                    html.Div([
                        html.H3(id="sum-left", style={"color": "#2ca02c"},  # Green
                                className="text-center text-lg font-bold"),
                        html.P("Left", className="text-center text-gray-600")
                    ]),
                    # Retired
                    html.Div([
                        html.H3(id="sum-retired", style={"color": "#ff7f0e"},  # Orange
                                className="text-center text-lg font-bold"),
                        html.P("In Retirement", className="text-center text-gray-600")
                    ]),
                ],
            ),

            # --- Monthly Trends ---
            html.Div(
                className="grid grid-cols-2 gap-6",
                children=[
                    html.Div([
                        html.H3("Monthly Trend: Active", className="text-center text-gray-800"),
                        dcc.Graph(id="active-monthly-trend-line")
                    ]),
                    html.Div([
                        html.H3("Monthly Trend: Left and Retired",
                                className="text-center text-gray-800"),
                        dcc.Graph(id="retired-left-monthly-trend-line")
                    ]),
                ]
            ),

            # --- Yearly Trends ---
            html.Div(
                className="grid grid-cols-2 gap-6",
                children=[
                    html.Div([
                        html.H3("Yearly Trend: Active", className="text-center text-gray-800"),
                        dcc.Graph(id="active-trend-line")
                    ]),
                    html.Div([
                        html.H3("Yearly Trend: Left and Retired",
                                className="text-center text-gray-800"),
                        dcc.Graph(id="retired-left-trend-line")
                    ]),
                ]
            ),
        ]
    )

### KPI Update Callback Explanation
This section describes the functionality of the callback function responsible for dynamically updating the KPI values in the HR Analytics Dashboard based on the selected year from the dropdown.
1. **Callback Definition**:
    - The callback is triggered when a value is selected in the `year-dropdown` component.
    - It updates the children of the three KPI elements:
        - **`kpi-avg-salary`**
        - **`kpi-avg-satisfaction`**
        - **`kpi-avg-absence`**

2. **Input Validation**:
    - The function first checks if a valid year is selected and whether the `Year` column exists in the dataset (`df`).
    - If no year is selected or the `Year` column is missing, all KPI values are set to default: `"No data"`.

3. **Data Filtering**:
    - The dataset (`df`) is filtered to include only rows where the `Year` matches the selected year.
    - If the filtered data is empty, the KPIs are set to `"No data"`.

4. **KPI Calculations**:
    - **Average Salary**:
The mean of the `Salary` column within the filtered data is calculated.
    - **Average Satisfaction**:
The mean of the `Satisfaction` column is calculated and formatted on a scale of 1 to 10.
    - **Average Sick Days (Illness)**:
        - Data is further filtered to include only rows where the `Absence Reason` is `"Illness"`.
        - The mean of the `Absence Days` column is calculated.

5. **Output Formatting**:
    - The results for each KPI are formatted into user-friendly strings:
        - Salary: Displayed in euros with two decimal places (e.g., `"3450.55 €"`).
        - Satisfaction: Presented on a 10-point scale with two decimal places (e.g., `"8.34 / 10"`).
        - Sick Days: Displayed in days with two decimal places (e.g., `"5.21 days"`).

    - If the calculated value is `NaN`, it is replaced with `"No data"`.

6. **Purpose**:
    - This callback ensures that the dashboard dynamically reflects the data for the selected year, providing insights into key metrics such as salary, satisfaction, and absence trends.


In [4]:
@app.callback(
    [Output("kpi-avg-salary", "children"),
     Output("kpi-avg-satisfaction", "children"),
     Output("kpi-avg-absence", "children")],
    Input("year-dropdown", "value")
)
def update_kpis(selected_year):
    if selected_year is None or "Year" not in df.columns:
        return "No data", "No data", "No data"

    # Filter the data based on the selected year
    filtered_data = df[df["Year"] == selected_year]
    if filtered_data.empty:
        return "No data", "No data", "No data"

    # Calculate average salary and satisfaction
    avg_salary = filtered_data["Salary"].mean()
    avg_satisfaction = filtered_data["Satisfaction"].mean()

    # Calculate average sick days:
    # Filter rows where the Absence Reason is "Illness" and sum "Absence Days"
    sick_days_data = filtered_data[filtered_data["Absence Reason"] == "Illness"]
    avg_absence = sick_days_data["Absence Days"].mean()

    # Return the KPIs formatted
    return (
        f"{avg_salary:.2f} €" if not pd.isna(avg_salary) else "No data",
        f"{avg_satisfaction:.2f} / 10" if not pd.isna(avg_satisfaction) else "No data",
        f"{avg_absence:.2f} days" if not pd.isna(avg_absence) else "No data"
    )

### KPI Titles Update Callback Explanation
This section explains the functionality of the callback function responsible for dynamically updating the titles of the KPI sections in the HR Analytics Dashboard.
1. **Callback Definition**:
    - The callback is triggered when a year is selected using the `year-dropdown` component.
    - It updates the titles of the following three KPI sections:
        - **`kpi-title-salary`**
        - **`kpi-title-satisfaction`**
        - **`kpi-title-absence`**

2. **Input Validation**:
    - If no year is selected (`selected_year` is `None`), default titles with the suffix `(No Data)` are displayed for all three KPIs.
Examples:
        - `"Salary (No Data)"`
        - `"Satisfaction (No Data)"`
        - `"Sick Days (No Data)"`

3. **Dynamic Title Updates**:
    - When a valid year is selected from the dropdown, the titles are updated to include the selected year in parentheses.
Examples:
        - `"Salary (2023)"`
        - `"Satisfaction (2023)"`
        - `"Sick Days (2023)"`

4. **Purpose**:
    - This callback helps maintain clarity and context in the dashboard by indicating the year for which the KPIs are being displayed.
    - It ensures a user-friendly experience by dynamically reflecting the selection in the titles.


In [5]:
@app.callback(
    [Output("kpi-title-salary", "children"),
     Output("kpi-title-satisfaction", "children"),
     Output("kpi-title-absence", "children")],
    Input("year-dropdown", "value")
)
def update_kpi_titles(selected_year):
    if selected_year is None:
        return "Salary (No Data)", "Satisfaction (No Data)", "Sick Days (No Data)"
    return (
        f"Salary ({selected_year})",
        f"Satisfaction ({selected_year})",
        f"Sick Days ({selected_year})"
    )

### Status Sums Update Callback Explanation
This section explains the callback function responsible for dynamically updating the yearly totals for **New Hires**, **Exits**, and **Retirements** in the HR Analytics Dashboard based on the selected year.
1. **Callback Definition**:
    - The callback is triggered when a year is selected from the `year-dropdown` component.
    - It updates the content for the following status-related elements:
        - **`sum-new-hires`**
        - **`sum-left`**
        - **`sum-retired`**

2. **Input Validation**:
    - The function ensures that all required columns (`Hiring Date`, `Exit Date`, `Status`, `Month`, `Employee_ID`) are available in the dataset (`df`).
    - If any of these columns are missing, it raises a `ValueError`.

3. **Data Preparation**:
    - The `Hiring Date` and `Exit Date` columns are converted to datetime format for accurate filtering. Any invalid or missing dates are handled using the `errors="coerce"` parameter.

4. **Default Case**:
    - If no year is selected (`selected_year` is `None`), the function returns default messages:
`"No year selected"` for all three metrics.

5. **Dynamic Calculations**:
The function dynamically filters and aggregates the data based on the selected year:
    - **New Hires**:
        - Filters rows where the `Hiring Date` falls within the selected year.
        - Calculates the number of unique employees hired during that year (`Employee_ID` count).

    - **Exits**:
        - Filters rows where:
            - The `Status` is `"Left"`.
            - The `Exit Date` falls within the selected year.

        - Counts the number of unique employees who left (`Employee_ID` count).

    - **Retirements**:
        - Filters rows where:
            - The `Status` is `"Retired"`.
            - The `Exit Date` falls within the selected year.

        - Counts the number of unique retirees (`Employee_ID` count).

6. **Output Results**:
    - The calculated totals are returned as formatted strings:
        - New Hires: `"X people"`
        - Exits: `"Y people"`
        - Retirements: `"Z people"`

    - If no employees meet the criteria for a particular metric, the output will display `"0 people"`.

7. **Purpose**:
    - This callback keeps the dashboard responsive to user selections by dynamically displaying key workforce changes for the selected year.
    - It provides insights into the hiring, turnover, and retirement trends within the organization.


In [6]:
@app.callback(
    [Output("sum-new-hires", "children"),
     Output("sum-left", "children"),
     Output("sum-retired", "children")],
    Input("year-dropdown", "value")
)
def update_status_sums(selected_year):
    # Ensure all required columns are available
    required_columns = ["Hiring Date", "Exit Date", "Status", "Month", "Employee_ID"]
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"The column '{col}' is missing in the dataset.")

    # Convert 'Hiring Date' and 'Exit Date' to datetime format
    df["Hiring Date"] = pd.to_datetime(df["Hiring Date"], errors="coerce", format="%Y-%m-%d")
    df["Exit Date"] = pd.to_datetime(df["Exit Date"], errors="coerce", format="%Y-%m-%d")

    # If no year is selected, return default values
    if not selected_year:
        return "No year selected", "No year selected", "No year selected"

    # FILTER: New Hires
    new_hires_df = df[
        (df["Hiring Date"].notna()) &
        (df["Hiring Date"].dt.year == selected_year)
        ]
    num_new_hires = new_hires_df["Employee_ID"].nunique()

    # FILTER: Exits
    left_df = df[
        (df["Status"] == "Left") &
        (df["Exit Date"].notna()) &
        (df["Exit Date"].dt.year == selected_year)
        ]
    num_exits = left_df["Employee_ID"].nunique()

    # FILTER: Retirements
    retired_df = df[
        (df["Status"] == "Retired") &
        (df["Exit Date"].notna()) &
        (df["Exit Date"].dt.year == selected_year)
        ]
    num_retired = retired_df["Employee_ID"].nunique()

    # Return the calculated results
    return (
        f"{num_new_hires} people",  # New Hires
        f"{num_exits} people",  # Exits
        f"{num_retired} people"  # Retirements
    )


### Active Monthly Trend Line Callback Explanation
This section describes the functionality of the callback responsible for dynamically updating the line chart that visualizes the monthly trend of active employees for the selected year.
1. **Callback Definition**:
    - The callback is triggered when a year is selected from the `year-dropdown` component.
    - It updates the `figure` property of the line chart identified as `active-monthly-trend-line`.

2. **Data Filtering**:
    - The dataset (`df`) is filtered to include only rows where:
        - The `Status` column is `"Active"`.
        - The `Year` column matches the selected year (`selected_year`).

    - This ensures the visualization accurately reflects the active employee trends for the specific year.

3. **Aggregation by Month**:
    - After filtering, the data is grouped by the `Month` column.
    - The size of each group (count of `Active` employees for each month) is calculated.
    - The resulting data contains two columns:
        - **`Month`**: Represents each month of the selected year.
        - **`Count`**: Total number of active employees in that month.

4. **Line Chart Creation**:
    - The `plotly.express.line` (`px.line`) function is used to create a line chart from the aggregated data with the following configurations:
        - **X-Axis**: `Month` (displayed in chronological order).
        - **Y-Axis**: `Count` (number of active employees).
        - **Title**: `"Active Trend"`.
        - **Markers**: Enabled to highlight individual data points on the line.
        - **Color**: Defined by a custom `color_discrete_map` for consistent color coding.

5. **Output**:
    - The generated line chart (`fig`) is returned as the output to be displayed in the dashboard.

6. **Purpose**:
    - This callback provides a clear visualization of workforce activity trends on a month-to-month basis for the selected year.
    - It enables users to easily detect patterns, fluctuations, or irregularities in the number of active employees throughout the year.


In [7]:
@app.callback(
    Output("active-monthly-trend-line", "figure"),
    Input("year-dropdown", "value")
)
def update_active_monthly_trend(selected_year):
    # Filter data for active employees in the selected year
    filtered_data = df[(df["Status"] == "Active") & (df["Year"] == selected_year)]
    filtered_data = filtered_data.groupby("Month").size().reset_index(name="Count")

    # Create a line plot
    fig = px.line(
        filtered_data,
        x="Month",
        y="Count",
        title="Active Trend",
        color_discrete_map=color_mapping,
        markers=True
    )
    return fig


### Monthly and Yearly Trend Line Callback Explanations
This section describes the series of callback functions responsible for visualizing trends related to active employees, as well as employees who have retired or left the organization. Each callback dynamically updates its respective chart based on the selected year.
#### **1. Monthly Retirement/Departure Trend Line**
This callback updates the line chart titled **"Retired/Left Trend"**, which displays the number of employees retiring or leaving each month for the selected year.
**Functionality**:
1. **Data Filtering**:
    - Filters the dataset (`df`) to include only rows where the `Status` is `"Left"` or `"Retired"` and the `Year` matches the `selected_year`.

2. **Data Aggregation**:
    - Groups the filtered data by `Month` and `Status`.
    - Counts the number of occurrences for each combination of `Month` and `Status`.

3. **Line Chart Creation**:
    - A plotly line chart (`px.line`) is created with:
        - **X-Axis**: `Month`
        - **Y-Axis**: `Count` (number of employees retired/left).
        - **Color**: Differentiates between `"Left"` and `"Retired"` using the `Status` column.
        - **Markers**: Enabled for better visualization of individual data points.

**Purpose**:
This visualization helps users analyze monthly fluctuations in employee departures (both voluntary and retirement) for the selected year.
#### **2. Yearly Active Employee Trend**
This callback updates the line chart titled **"Active - Yearly Trend"**, showing the total number of active employees over the years.
**Functionality**:
1. **Data Filtering**:
    - Filters the dataset to include only rows where the `Status` is `"Active"`.

2. **Unique Employee-Year Combinations**:
    - Removes duplicate records to ensure each employee is only counted once per year.

3. **Data Aggregation**:
    - Groups the filtered data by `Year`.
    - Counts the number of unique `Employee_ID` entries for each year.

4. **Line Chart Creation**:
    - A plotly line chart is generated with:
        - **X-Axis**: `Year`
        - **Y-Axis**: `Count` (number of active employees).
        - **Title**: `"Active - Yearly Trend"`
        - **Markers**: Enabled for better data point representation.

**Purpose**:
This yearly trend line helps analyze overall changes in the organization's active workforce over time.
#### **3. Yearly Retirement/Departure Trend**
This callback updates the line chart titled **"Yearly Trends (Left/Retired)"**, which illustrates yearly totals for employees retiring or leaving the organization.
**Functionality**:
1. **Data Filtering**:
    - Filters the dataset to include employees with `Status` equal to `"Left"` or `"Retired"`.

2. **Data Aggregation**:
    - Groups the filtered data by `Year` and `Status`.
    - Calculates the count of employees for each combination of `Year` and `Status`.

3. **Line Chart Creation**:
    - A line chart is created with:
        - **X-Axis**: `Year`
        - **Y-Axis**: `Count` (number of employees left/retired).
        - **Color**: Differentiates between `"Left"` and `"Retired"` using the `Status` column.
        - **Title**: `"Yearly Trends (Left/Retired)"`
        - **Markers**: Enabled for easier interpretation of data points.

**Purpose**:
This visualization allows users to analyze trends in employee departures and retirement rates across multiple years.
### General Notes
- The callbacks dynamically process the dataset based on user input (`selected_year`) to provide focused visual insights into employee trends.
- Custom color schemes (via `color_discrete_map`) ensure charts are visually consistent across the dashboard.


In [8]:
@app.callback(
    Output("retired-left-monthly-trend-line", "figure"),
    Input("year-dropdown", "value")
)
def update_retired_left_monthly_trend(selected_year):
    # Filter data for employees with status "Left" or "Retired" in the selected year
    filtered_data = df[(df["Status"].isin(["Left", "Retired"])) & (df["Year"] == selected_year)]
    filtered_data = filtered_data.groupby(["Month", "Status"]).size().reset_index(name="Count")

    # Create a line plot
    fig = px.line(
        filtered_data,
        x="Month",
        y="Count",
        color="Status",
        title="Retired/Left Trend",
        color_discrete_map=color_mapping,
        markers=True
    )
    return fig


In [9]:
@app.callback(
    Output("active-trend-line", "figure"),
    Input("year-dropdown", "value")
)
def update_active_trend(selected_year):
    # Filter for active employees
    filtered_data = df[df["Status"] == "Active"]

    # Keep only unique combinations of Employee_ID and Year
    unique_data = filtered_data.drop_duplicates(subset=["Employee_ID", "Year"])

    # Group by Year and count unique entries
    trend_data = unique_data.groupby("Year").size().reset_index(name="Count")

    # Create the visualization
    fig = px.line(
        trend_data,
        x="Year",
        y="Count",
        title="Active - Yearly Trend",
        color_discrete_map={"Active": "#1f77b4"},
        markers=True
    )
    return fig

In [10]:
@app.callback(
    Output("retired-left-trend-line", "figure"),
    Input("year-dropdown", "value")
)
def update_retired_left_trend(selected_year):
    # Filter data for employees with status "Left" or "Retired"
    filtered_data = df[df["Status"].isin(["Left", "Retired"])]

    # Group by Year and Status, and count occurrences
    filtered_data = filtered_data.groupby(["Year", "Status"]).size().reset_index(name="Count")

    # Create a line plot
    fig = px.line(
        filtered_data,
        x="Year",
        y="Count",
        color="Status",
        title="Yearly Trends (Left/Retired)",
        color_discrete_map=color_mapping,
        markers=True
    )
    return fig

### Render Page 2 and Interactive Analysis Callback Explanation
This section explains the functionality included in rendering the second page of the analytics dashboard and showcases interactive analysis through visualization updates in response to user input.
#### **1. `render_page_2` Function**
This function is responsible for rendering the layout of the second page of the dashboard. It includes the following components:
1. **Scatterplot Visualization**:
    - A scatter plot is created to show the relationship between **Salary** and **Satisfaction**, with the `Status` field used for color coding.
    - The plot's attributes include:
        - **X-Axis**: `Salary`.
        - **Y-Axis**: `Satisfaction`.
        - **Color**: Differentiates between employee `Status` (e.g., `"Active"`, `"Retired"`, `"Left"`) using a custom `color_discrete_map`.
        - **Title**: `"Satisfaction vs. Salary by Status"`.

2. **Interactive Analysis Section**:
    - A **Dropdown** (`dcc.Dropdown`) is provided to allow users to select a feature they want to analyze further.
    - **Graph Placeholder**: An empty `dcc.Graph` component is used to display results dynamically based on the dropdown selection. It will be updated by the callback function.

**Purpose**:
This function creates a visually detailed and interactive page where users can explore relationships between employee satisfaction, salary, and selected features for further analysis.
#### **2. `update_interactive_plot` Callback**
This callback updates the content of the **interactive graph** based on the selected feature from the dropdown.
**Functionality**:
1. **Input Validation**:
    - Checks whether the selected feature (`selected_feature`) exists in the dataframe (`df`).
    - If the feature is invalid or missing, it returns an empty plot (`px.scatter`) with a title: `"No valid data available"`.

2. **Dynamic Histogram Creation**:
    - A histogram is generated using the selected feature:
        - **X-Axis**: The feature chosen by the user (e.g., `Age`, `Salary`).
        - **Y-Axis**: Frequency distribution of data, optionally normalized to **percent (`histnorm="percent"`)**.
        - **Color**: Groups data by `Status` for better comparative analysis.
        - **Barmode**: Displays the histogram in **grouped mode**, which separates the data by `Status`.

    - The title is dynamically set as `"Frequency Distribution: <Selected Feature>"` for clarity.

**Output**:
The returned histogram plot is displayed in the `dcc.Graph` component (`interactive-plot`) based on user interaction.
### Features and Benefits Summary
- **Scatter Plot Analysis**:
    - Provides insights into how employee **Satisfaction** correlates to **Salary**.
    - Highlights differences based on employee status categories (e.g., "Active", "Retired").

- **Dynamic Dropdown-Controlled Visualization**:
    - Allows flexible analysis by enabling users to select specific features (like `Age` or `Salary`) for detailed distribution exploration.
    - Visualizes the data distribution for selected features grouped by employee `Status`, offering deeper comparative insights.

- **User-Friendly Interactivity**:
    - By combining static plots with dropdown-driven dynamic updates, the page supports both general exploration and detailed, focused analysis tailored to user preferences.


In [11]:
def render_page_2():
    scatter_fig = px.scatter(df, x="Salary", y="Satisfaction", color="Status",
                                 title="Satisfaction vs. Salary by Status", color_discrete_map=color_mapping
                                 )
    return html.Div([
        html.H2("Analytics & Details", className="text-center text-blue-600 font-bold text-2xl mb-6"),

        # Scatterplot: Satisfaction vs Salary
        dcc.Graph(figure=scatter_fig, className="rounded shadow-lg"),

        # Dropdown for interactive analysis
        html.Div(
            className="mt-6",
            children=[
                html.H3("Interactive Analysis", className="text-center text-gray-700 text-lg"),
                dcc.Dropdown(
                    id="analysis-dropdown",
                    options=[
                        {"label": "Age", "value": "Age"},
                        {"label": "Salary", "value": "Salary"},
                    ],
                    value="Age",
                ),
                dcc.Graph(id="interactive-plot"),
            ]
        )
    ])


In [12]:
@app.callback(
    Output("interactive-plot", "figure"),  # Output for the interactive graphic
    Input("analysis-dropdown", "value")  # Input from the dropdown
)
def update_interactive_plot(selected_feature):
    # Ensure the selection is valid
    if selected_feature not in df.columns:
        return px.scatter(title="No valid data available")

    # Create a histogram based on the selected feature from the dropdown
    fig = px.histogram(
        df,
        x=selected_feature,  # X-axis based on the selected column
        title=f"Frequency Distribution: {selected_feature}",  # Title of the plot
        color="Status",  # Color based on 'Status'
        barmode="group",  # Grouped display of bars
        histnorm="percent"  # Normalize to percent (optional)
    )
    return fig


### Page 3: Correlation Matrix Visualization
This section explains the functionality used to render the **Correlation Matrix** on the third page of the dashboard. It provides a clear and detailed view of the relationships between numerical features in the dataset while omitting irrelevant or redundant data (like self-correlations).
#### **1. Render Correlation Matrix**
The function `render_page_3_correlation_matrix` handles the creation and display of the Correlation Matrix. Below are its key features and steps:
##### **Step 1: Filtering Numeric Columns**
- The function begins by selecting only numeric columns from the dataset (`df`). This ensures the correlation matrix is calculated exclusively on features that support mathematical correlation.
- **Validation**:
    - If the dataset doesn't contain any numeric columns, the function returns a message informing the user that no correlation analysis is possible.

##### **Step 2: Computing the Correlation Matrix**
- The correlation matrix is calculated using the `.corr()` method, which provides pairwise correlation coefficients between numeric columns.
- **Key Considerations**:
    - High correlation values (close to +1 or -1) indicate strong positive or negative relationships, while values near 0 signify weak or no correlation.

##### **Step 3: Removing the Diagonal**
- Since the diagonal of the matrix contains self-correlation values (always 1.0), these values are masked (hidden) using a NumPy identity matrix (`np.eye`). This improves visual clarity by emphasizing relationships between different columns.

##### **Step 4: Creating the Heatmap**
- A high-resolution heatmap is generated using **Seaborn's `heatmap`** function:
    - **Diagonal Removed**: Ensures focus is only on cross-variable relationships.
    - **Colors**: A `"coolwarm"` colormap is used to distinguish strong correlations (positive in red, negative in blue).
    - **Annotations**: Each correlation coefficient is displayed directly on the heatmap, rounded to two decimal places for readability.
    - **Styling**:
        - Font sizes are adjusted for axis labels and the plot title to enhance usability.
        - The plot is square-shaped, with a color bar scaled for clarity (`shrink=0.6`).

    - The figure is configured to a size of 20x20 inches with a DPI of 300 for high resolution, ensuring professional-looking output.

##### **Step 5: Encoding the Heatmap**
- The heatmap is saved as a PNG image into a `BytesIO` buffer.
- The image is encoded using Base64, converting it into a format that can be directly displayed in the Dash app as an `<img>` component.

##### **Step 6: Embedding the Heatmap in Dash**
- The correlation matrix image is embedded in the layout of the page as an `<img>` tag:
    - Fully responsive, scaled to fit the available screen space (`width: 100%`).

### Features Summary
1. **Dynamic Validation**:
    - Ensures end-users only see meaningful content. If the dataset has no numeric columns, an informative message is displayed instead of a heatmap.

2. **Diagonal-Free Correlation Heatmap**:
    - Self-correlations are removed, allowing users to focus on relationships between distinct features.

3. **High-Resolution Visualization**:
    - Clear and professional heatmap generated using **Seaborn**, highlighting both positive and negative correlations with a visually appealing colormap.

4. **Dash Integration**:
    - The heatmap is integrated seamlessly into the dashboard layout as a responsive image, offering a user-friendly and visually informative experience.

### Purpose of the Correlation Matrix
The correlation matrix enables users to:
- Detect and quantify linear relationships between numerical features.
- Identify features with the strongest or weakest correlations.
- Explore patterns that can guide further statistical modeling or decision-making.

In [13]:
def render_page_3_correlation_matrix():
    # Keep only numeric columns
    numeric_df = df.select_dtypes(include=["number"])
    if numeric_df.empty:
        return html.Div([
            html.H2("Correlation Matrix", className="text-center text-blue-600 font-bold text-2xl mt-6 mb-6"),
            html.P("The dataset does not contain any numeric columns that can be correlated.",
                   className="text-center text-gray-600 text-lg"),
        ])

    # Calculate the correlation matrix
    correlation_matrix = numeric_df.corr()

    # Remove the main diagonal using masking
    mask = np.eye(len(correlation_matrix), dtype=bool)
    correlation_matrix_no_diag = correlation_matrix.mask(mask)

    # High-resolution heatmap using Seaborn
    plt.figure(figsize=(20, 20), dpi=300)
    heatmap = sns.heatmap(
        correlation_matrix_no_diag,
        annot=True,
        fmt=".2f",
        cmap="coolwarm",
        square=True,
        cbar_kws={"shrink": 0.6},
        linewidths=0.5,
        mask=mask
    )
    plt.title("Correlation Matrix Without Diagonal", fontsize=20, pad=30)
    plt.xticks(rotation=45, fontsize=12, ha="right")
    plt.yticks(fontsize=12)

    # Encode the Matplotlib plot as a Base64 image
    buf = BytesIO()
    plt.savefig(buf, format="png", bbox_inches="tight")  # Trim edges
    buf.seek(0)
    encoded_image = base64.b64encode(buf.read()).decode("utf-8")
    buf.close()

    # Embed Matplotlib image in Dash
    return html.Div([
        html.H2("Correlation Matrix Without Diagonal",
                className="text-center text-blue-600 font-bold text-2xl mt-4 mb-4"),
        html.Img(src=f"data:image/png;base64,{encoded_image}", style={"width": "100%", "height": "auto"}),
        # Scale image
    ])


### Page 4: Data Upload & Critical Employee Identification
This section explains the functionality for **uploading datasets** and **identifying critical employees** using a predictive model. The page allows users to upload a file, select a model, set a threshold for turnover probability, and visualize results.
### **1. Render Page 4**
The `render_page_4` function creates the layout for the **Data Upload & Update** page. It includes the following components:
#### **Components**
1. **Page Title**:
    - Displays the title `"Data Upload & Update"` to introduce the page.

2. **File Upload Section**:
    - **Drag-and-Drop File Upload Component**:
        - Allows users to upload a file in `.csv` format.
        - Displays a placeholder encouraging users to drag & drop or select a file from their device.

3. **Model Selection**:
    - A dropdown where users can select a predictive model (e.g., `"XGBoost"`).

4. **Threshold Input**:
    - An input box for users to specify the **threshold** percentage for identifying critical employees based on turnover probability (default: 10%).

5. **Feedback and Outputs**:
    - **Upload Feedback**: Displays confirmation or errors related to file uploads.
    - **Critical Employee Table**: Displays the top-ranking critical employees by risk.
    - **Status Message**: Provides information about processing results.

6. **Default Behavior Note**:
    - A note informing users that a default file will be used if no file is uploaded.

### **2. File Upload and Processing Callbacks**
#### **Callback 1: File Upload Feedback**
This callback provides user-friendly feedback for the file upload process.
**Functionality**:
- Listens for any uploaded file (`contents`) along with the filename.
- Validates the file type and parses the content if it is a valid `.csv` file.
- Feedback messages include:
    - **Success**: Confirms the file was uploaded successfully.
    - **Invalid File Type**: Informs the user that only `.csv` files are allowed.
    - **Error**: Displays any issues encountered during file processing.

#### **Callback 2: Process and Display Data**
This callback handles processing the uploaded file, selecting the model, applying the threshold, and displaying results dynamically.
**Inputs**:
- **Uploaded File**: Reads the content of the uploaded file in `.csv` format.
- **Model Selection**: Determines which predictive model will be used for identifying critical employees.
- **Threshold Input**: Applies a percentage cutoff for identifying high-risk employees.

**Outputs**:
1. **Upload Feedback**:
    - Communicates file processing results or errors.

2. **Critical Employee Table**:
    - Displays the top 15 high-risk employees in a tabular format, including:
        - Year, Month, Employee ID, Name, Position, Age, Status, and Turnover Probability.

3. **Upload Status**:
    - Summarizes the number of identified critical employees or errors during the process.

**Steps**:
1. **File Loading**:
    - Decodes and loads the uploaded dataset into a Pandas DataFrame.
    - Temporarily saves the processed file for further analysis.

2. **Model Validation and Loading**:
    - Checks whether a valid model (e.g., `"XGBoost"`) is selected.
    - Loads the chosen predictive model from the predefined directory.

3. **Critical Employee Identification**:
    - Processes the data and uses the model to calculate turnover probabilities for all employees.
    - Filters the employees based on the given threshold to identify those at the highest risk of turnover.

4. **Results Display**:
    - Returns the top 15 at-risk employees, formatted into a responsive Dash DataTable for easy analysis.
    - Each column is styled for better readability (e.g., light blue header for emphasis).

5. **Error Handling**:
    - Catches and displays any errors encountered during file processing, model loading, or predictions.

### **3. Critical Employee Processing**
The `process_and_identify_critical_employees` function performs the core logic for identifying at-risk employees. Below is an overview of its workflow:
**Step 1: Data Preprocessing**
- **Dataset Loading and Cleaning**:
    - Handles reading, cleaning, and preparing the dataset for predictions.

- **Feature Transformation**:
    - Applies preprocessing steps, including scaling and encoding, for compatibility with the model.

**Step 2: Model Loading**
- Loads the predictive model (e.g., `"XGBoost"`) and associated feature names from the specified directory.

**Step 3: Model Predictions**
- Applies the trained model to the preprocessed data to generate turnover probability scores.

**Step 4: Filtering Results**
- Filters employees whose turnover probabilities exceed the threshold.
- Ranks employees by risk and selects the top 15 candidates for display.

**Outputs**:
- A DataFrame containing all critical employees.
- A filtered DataFrame with the top 15 employees by turnover probability.

### **Features and Benefits**
1. **Dynamic File Upload**:
    - Ensures easy integration of custom datasets for personalized analysis.

2. **Model Flexibility**:
    - Users can select their predictive model, allowing flexibility for future expansions with additional algorithms.

3. **Threshold-Based Insights**:
    - User-defined thresholds enable customization of risk levels, tailoring the identification process to organizational needs.

4. **Detailed Results Table**:
    - Interactive Dash DataTable allows users to view and analyze the most critical employees efficiently.

5. **Error Handling**:
    - Robust error messages and validation keep users informed about issues in real time.

### Purpose of the Page
The **Data Upload & Update** page empowers organizations to:
- Analyze employee data dynamically by uploading new or updated datasets.
- Harness predictive models to identify employees most at risk of turnover.
- Enable decision-makers to focus on high-risk employees for targeted interventions.

In [14]:
def render_page_4():
    return html.Div(
        children=[
            html.H2(
                "Data Upload & Update",
                className="text-center text-blue-600 font-bold text-2xl mb-6",
            ),
            html.Div(
                className="mt-4 mb-6",
                children=[
                    html.H3("Upload CSV File:", className="text-lg text-gray-700"),
                    dcc.Upload(
                        id="upload-data",
                        children=html.Div(
                            ["Drag and drop file here or ", html.A("Select File")]
                        ),
                        style={
                            "width": "100%",
                            "height": "60px",
                            "lineHeight": "60px",
                            "borderWidth": "1px",
                            "borderStyle": "dashed",
                            "borderRadius": "5px",
                            "textAlign": "center",
                            "margin": "10px",
                        },
                        multiple=False,
                    ),
                    html.Div(id="upload-feedbacks", className="mt-4 text-lg text-gray-700"),
                    html.H3("Select Model:", className="mt-6 text-lg text-gray-700"),
                    dcc.Dropdown(
                        id="model-dropdown",
                        options=[
                            {"label": "XGBoost", "value": "XGBoost"},
                        ],
                        placeholder="Choose a Model",
                        className="mt-2",
                    ),
                    html.H3("Threshold for Turnover Probability (%):",
                            className="mt-6 text-lg text-gray-700"),
                    dcc.Input(
                        id="threshold-input",
                        type="number",
                        value=10,
                        placeholder="Enter Threshold",
                        className="mt-2",
                    ),
                    html.Div(id="upload-status", className="mt-4 text-lg text-gray-700"),
                    html.Div(id="critical-employees-table", className="mt-8", style={"marginBottom": "40px"}
                             ),
                    html.Div(
                        children=[
                            html.H3(
                                "Note: If no file is uploaded, the default file will be used.",
                                className="text-md text-gray-500",
                            )
                        ]
                    ),
                ],
            ),
        ],
    )


In [15]:
@app.callback(
    Output("upload-feedback", "children"),
    Input("upload-data", "contents"),
    State("upload-data", "filename"),
)
def update_dataset(content, filename):
    global df  # Access the global variable for the DataFrame

    if content is not None:
        # Decode content and convert it into a Pandas DataFrame
        content_type, content_string = content.split(",")
        decoded = base64.b64decode(content_string)

        try:
            # Check if the uploaded file is a CSV file
            if filename.endswith(".csv"):
                df = pd.read_csv(io.StringIO(decoded.decode("utf-8")), low_memory=False)
                return f"The file '{filename}' was successfully uploaded!"
            else:
                return "Please upload CSV files only."
        except Exception as e:
            return f"Error reading the file: {str(e)}"
    else:
        return "No file uploaded."


In [16]:
# Callback for file upload and processing
@app.callback(
    [
        Output("upload-feedbacks", "children"),  # Feedback to the user
        Output("critical-employees-table", "children"),  # Table showing critical employees
        Output("upload-status", "children"),  # Upload status message
    ],
    [
        Input("upload-data", "contents"),  # Uploaded file contents
        Input("model-dropdown", "value"),  # Selected model from the dropdown
        Input("threshold-input", "value"),  # Threshold value for critical employees
    ],
    [State("upload-data", "filename")],  # Filename of the uploaded file
)
def process_and_display_critical_employees(contents, selected_model, threshold, filename):
    # Check if any file was uploaded
    if not contents:
        return "Please upload a file.", None, "No file uploaded."

    # Check if a model was selected
    if not selected_model:
        return "Please select a model.", None, "No model selected."

    try:
        # ** Step 1: Load the file **
        content_type, content_string = contents.split(",")
        decoded = base64.b64decode(content_string)
        uploaded_df = pd.read_csv(io.StringIO(decoded.decode("utf-8")))

        # Temporary storage path for the uploaded file
        temp_path = "../data/uploaded_temp.csv"
        uploaded_df.to_csv(temp_path, index=False)  # Save the file temporarily

        file_path = temp_path

        # Process and identify critical employees
        critical_employees, top_15_employees = process_and_identify_critical_employees(
            file_path,  # Input file path
            save_filtered_path=None,  # Path to save filtered data
            models_dir="models",  # Directory where the model is stored
            model_file_name="xgboost_model.pkl",  # Model file name
            feature_names_file="models/lightgbm_feature_names.pkl",  # Feature names file
            threshold=0.0  # Threshold for critical employees
        )

        # Check if any critical employees were found
        if critical_employees is None or critical_employees.empty:
            return "No critical employees found.", None, "No critical employees."

        # Define columns to display in the table
        columns_to_display = [
            'Year', 'Month', 'Employee_ID', 'Name', 'Position',
            'Age', 'Status', 'Turnover_Probability'
        ]

        # Filter the DataFrame to include only the selected columns
        filtered_top_15_employees = top_15_employees[columns_to_display]

        # Create a DataTable using Dash
        table = dash_table.DataTable(
            id="critical-employees-results",
            columns=[{"name": col, "id": col} for col in filtered_top_15_employees.columns],
            data=filtered_top_15_employees.to_dict("records"),
            style_table={"overflowX": "auto"},
            style_cell={"textAlign": "left", "padding": "5px"},
            style_header={"backgroundColor": "lightblue", "fontWeight": "bold"},
            page_size=15,  # Maximum rows per page
        )

        return f"File '{filename}' successfully processed.", table, f"{len(critical_employees)} critical employees found."

    except Exception as e:
        # Handle any errors that occur during file upload or processing
        return f"An error occurred: {str(e)}", None, "Error during processing."



In [17]:
def process_and_identify_critical_employees(
        file_path,  # Input file
        save_filtered_path=None,  # Path to save filtered data
        models_dir="Models",  # Directory where the model is stored
        model_file_name="xgboost_model.pkl",  # Name of the model file
        feature_names_file="Models/lightgbm_feature_names.pkl",  # File containing feature names
        threshold=0.0  # Threshold for identifying critical employees
):
    """
    This function handles data preprocessing, model loading, and identification
    of critical employees based on a predictive model.

    Args:
        file_path (str): Path to the input file (CSV).
        save_filtered_path (str): Path to save the filtered input file.
        models_dir (str): Directory where the trained model is stored.
        model_file_name (str): Name of the file in which the model is stored.
        feature_names_file (str): File containing feature names for interpretation.
        threshold (float): Threshold for identifying critical employees.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: DataFrame with critical employees and the top 15 employees by risk.
    """
    try:
        # Load and clean data
        df = load_dataset(file_path, save_filtered_path)
        df_cleaned = clean_dataset(df)

        # Preprocess data
        df_processed, X_transformed, X_resampled, y_resampled, preprocessor = preprocess_data(df_cleaned)

        # Create model path and load the model
        model_file = os.path.join(models_dir, model_file_name)
        try:
            model = joblib.load(model_file)  # Load the model
            print("The model was successfully loaded.")
        except FileNotFoundError:
            print(f"Error: The model file '{model_file}' was not found.")
            return None, None
        except Exception as e:
            print(f"Error while loading the model: {e}")
            return None, None

        # Identify critical employees
        critical_employees, top_15_employees = get_critical_employees(
            model, X_transformed, df_processed, feature_names_file=feature_names_file, threshold=threshold
        )

        # Output results
        print(
            f"Top 15 Employees:\n{top_15_employees[['Employee_ID', 'Name', 'Turnover_Probability']]}"
            # Update column names
        )

        return critical_employees, top_15_employees
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

In [18]:
# Start the app in "inline" mode
# Automatically open in the web browser
app.run(debug=True, open_browser=True)